In [2]:
from __future__ import print_function
import numpy as np 
import pandas as pd 
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
import IPython.display as ipd
import seaborn as sns
import warnings
import os
import torch
from torch import nn, optim
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

In [3]:
audio_features = pd.read_pickle('/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/preprocess_info/feature_vectors.pkl')

In [4]:
type(audio_features)

dict

In [5]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'exc': 2,
                'sad': 3,
                'fru': 4,
                'fea': 5,
                'sur': 6,
                'neu': 7,
                'xxx': 8,
                'oth': 8,
                'dis': 8}

In [6]:
audio_features.keys()

dict_keys(['wav_file', 'label', 'mfccs', 'spec_db'])

In [7]:
audio_features['spec_db'][0].shape


(128, 84)

In [8]:
b=[]
for i in range(len(audio_features['mfccs'])):
    a=audio_features['spec_db'][i].shape[1]
    b.append(a)
len(b)

10039

In [9]:
c=[]
for i in range(len(audio_features['label'])):
    a=audio_features['spec_db'][i]
    c.append(a)
len(c)

10039

In [10]:
labelsl = []
mfccsl = []
spec_dbl = []
for i in range(len(audio_features['label'])):
    if (audio_features['label'][i]) in [0, 1, 3, 7]:
        labelsl.append(audio_features['label'][i])
        mfccsl.append(audio_features['spec_db'][i])
        spec_dbl.append(audio_features['spec_db'][i])

In [11]:
len(mfccsl)

4490

In [12]:
f=[]
for i in range(len(mfccsl)):
    a = mfccsl[i].shape[1]
    f.append(a)
np.median(np.array(f))

152.0

In [13]:
# #num=160
# #padd=num-119
# #padd=padd/2
# #padd=padd+1
# #a=np.pad(np.array(mfccsl[0]),[(0,0),(int(padd),int(padd))],mode='constant')
# #garbage=a[:,:num]
# #garbage.shape

# finalmfccs = []
# for i in range(len(mfccsl)):
#     a = mfccsl[i]
#     if a.shape[1]<num:
#         padd = num - a.shape[1]
#         padd = padd / 2
#         padd = padd + 1
#         a = np.pad(np.array(a), [(0,0), (int(padd), int(padd))], mode = 'constant')
#     garbage = a[:, :num]
#     finalmfccs.append(garbage)

In [14]:
# np.array(finalmfccs).shape

In [15]:
# uncomment if running 1st time
# for i in range(len(labelsl)):
#     if labelsl[i] == 3:
#         labelsl[i] = 2
#     if labelsl[i] == 7:   # Neutral
#         labelsl[i] = 3
# # 0 is anger, 1 is happiness, 3 is sadness, 7 is neutral
# # Updated 0 is anger, 1 is happiness, 2 is sadness and 3 is neutral

In [16]:
Counter(labelsl)

Counter({7: 1708, 0: 1103, 3: 1084, 1: 595})

In [17]:
# print(np.array(finalmfccs).shape)     # The Final Pre-processed MFCCS
print(len(labelsl))                   # The Final Labels

4490


In [18]:
X_train, X_test, y_train, y_test = train_test_split(spec_dbl, labelsl, test_size=0.2)

## Manually splitting
#dataset_split_ratio = int(len(labelsl) * 0.80) 
#train_finalmfccs = finalmfccs[:dataset_split_ratio]
#train_finallabels = labelsl[:dataset_split_ratio]
#print(np.array(train_finalmfccs).shape)
#
#test_finalmfccs = finalmfccs[dataset_split_ratio:]
#test_finallabels = labelsl[dataset_split_ratio:]
#print(np.array(test_finalmfccs).shape)

In [19]:
print(X_train[1].shape) # (128, 75)
print(y_train[5]) # (128, 75) # emotion
#(128, 225)

(128, 100)
0


In [20]:
Counter(y_train)

Counter({0: 874, 7: 1376, 3: 867, 1: 475})

In [21]:
Counter(y_test)

Counter({0: 229, 7: 332, 1: 120, 3: 217})

In [22]:
batch_size = 256

In [23]:
def collate_fn(batch):
        # Determine the maximum length of the spectrograms in the batch
        max_len = max(spec.shape[1] for spec, label in batch)

        # Pad the spectrograms with zeros to the maximum length
        padded_specs = []
        for spec, label in batch:
            num_cols = spec.shape[1]
            padding = torch.zeros((128, max_len - num_cols))
            padded_specs.append(torch.cat([spec, padding], dim=1))

        # Concatenate the padded spectrograms into a tensor
        specs_tensor = torch.stack(padded_specs, dim=0)

        # Convert the labels to PyTorch tensor
        labels_tensor = torch.tensor([label for spec, label in batch])
        # Create a list of filenames
        # filenames_list = [filename for spec, label, filename in batch]

        return specs_tensor, labels_tensor#, filenames_list

In [24]:
class Dataset(Dataset):
    def __init__(self, X_train,y_train):
        
        self.spectrogram = X_train
        self.labels = y_train
        # self.filename = dataset['wav_file']

    def __len__(self):
        return len(self.spectrogram)

    def __getitem__(self, idx):
        return torch.tensor(self.spectrogram[idx]), torch.tensor(self.labels[idx])#, self.filename[idx]


train_dataset = Dataset(X_train,y_train)
# class AudioDataset_train(Dataset):
#     """
#     IEMOCAP Dataset
#     """
#     def __init__(self):
#         self.len = len(X_train)
#         self.x_data = torch.from_numpy(np.array(X_train))
#         self.y_data = torch.from_numpy(np.array(y_train))
        
#     def __getitem__(self, index):
#         return self.x_data[index], self.y_data[index]
    
#     def __len__(self):
#         return self.len
    
# traindataset = AudioDataset_train()

trainloader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         shuffle = True,collate_fn=collate_fn)

valdataset = Dataset(X_test,y_test)
valloader = DataLoader(dataset = valdataset,
                        batch_size = batch_size,collate_fn=collate_fn)

In [25]:
for idx, batch in enumerate(trainloader):
    print(batch[1].shape)
    break

torch.Size([256])


In [26]:
# class AudioDataset_test(Dataset):
#     def __init__(self, X_test):
        
#         self.spectrogram = X_test
#         # self.labels = y_train
#         # self.filename = dataset['wav_file']

#     def __len__(self):
#         return len(self.spectrogram)

#     def __getitem__(self, idx):
#         return torch.tensor(self.spectrogram[idx])#, torch.tensor(self.labels[idx])#, self.filename[idx]
    


In [27]:
len(valloader)

4

In [28]:
# testiter = iter(testloader)
# features, labels = next(testiter)
# features.shape, labels.shape

In [29]:
# features.type()

In [30]:
keep_prob = 0.5

In [31]:
# class CNN(torch.nn.Module):

#     def __init__(self):
#         super(CNN, self).__init__()
#         # L1 ImgIn shape=(?, 224, 224, 3)
#         #    Conv     -> (?, 224, 224, 16)
#         #    Pool     -> (?, 112, 112, 16)
#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2, stride=2),
#             torch.nn.Dropout(p=1 - keep_prob))
#         # L2 ImgIn shape=(?, 112, 112, 16)
#         #    Conv      ->(?, 112, 112, 32)
#         #    Pool      ->(?, 56, 56, 32)
#         self.layer2 = torch.nn.Sequential(
#             torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2, stride=2),
#             torch.nn.Dropout(p=1 - keep_prob))
#         # L3 ImgIn shape=(?, 56, 56, 32)
#         #    Conv      ->(?, 56, 56, 64)
#         #    Pool      ->(?, 28, 28, 64)
#         self.layer3 = torch.nn.Sequential(
#             torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2, stride=2),
#             torch.nn.Dropout(p=1 - keep_prob))
        
#         # L4 ImgIn shape=(?, 28, 28, 64)
#         #    Conv      ->(?, 28, 28, 16)
#         #    Pool      ->(?, 14, 14, 16)
#         self.layer4 = torch.nn.Sequential(
#             torch.nn.Conv2d(64, 16, kernel_size=1, stride=1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2, stride=2),
#             torch.nn.Dropout(p=1 - keep_prob))

#         # L4 FC 14x14x16 inputs -> 512 outputs
#         self.fc1 = torch.nn.Linear(14 * 14 * 16, 512, bias=True)
#         torch.nn.init.xavier_uniform(self.fc1.weight)
# #         self.layer4 = torch.nn.Sequential(
# #             self.fc1,
# #             torch.nn.ReLU(),
# #             torch.nn.Dropout(p=1 - keep_prob))
#         # L5 Final FC 1024 inputs -> 512 outputs
        
#         self.fc2 = torch.nn.Linear(512, 4, bias=True)
#         torch.nn.init.xavier_uniform_(self.fc2.weight) # initialize parameters
#         # L6 Final FC 512 inputs -> 4 outputs
# #         self.fc3 = torch.nn.Linear(512, 4, bias=True)
# #         torch.nn.init.xavier_uniform_(self.fc3.weight) # initialize parameters

#         self.dropout = nn.Dropout(p=0.3)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = self.layer4(out)
# #         print(out.size())
#         out = out.view(out.size(0), -1)
# #         print(out.size())# Flatten them for FC
#         out = self.fc1(out)
#         out = self.dropout(out)
#         out = self.fc2(out)
# #         out = self.fc3(out)
#         return out


In [38]:
keep_prob =0.5
class Crude_Diag(nn.Module):
    def __init__(self, in_features):
        super(Crude_Diag, self).__init__()
        
        # Set a fixed seed for reproducibility
        torch.manual_seed(0)
        
        # Initialize scaling factors and weight matrix
        scaling_factors = torch.rand(in_features)
        weight = torch.diag(scaling_factors)
        
        # Modify weight matrix to set non-diagonal elements to zero
        with torch.no_grad():
            for i in range(in_features):
                for j in range(in_features):
                    if i != j:
                        weight[i, j] = 0.0
                        weight[i, j].requires_grad = False
        
        # Define linear layer with diagonal weight matrix
        self.linear = nn.Linear(in_features, in_features, bias=False)
        self.linear.weight = nn.Parameter(weight)
        
    def forward(self, x):
        x = self.linear(x)
        return x

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 16)
        #    Pool     -> (?, 112, 112, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))
        # L2 ImgIn shape=(?, 112, 112, 16)
        #    Conv      ->(?, 112, 112, 32)
        #    Pool      ->(?, 56, 56, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))
        # L3 ImgIn shape=(?, 56, 56, 32)
        #    Conv      ->(?, 56, 56, 64)
        #    Pool      ->(?, 28, 28, 64)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))
        
        # L4 ImgIn shape=(?, 28, 28, 64)
        #    Conv      ->(?, 28, 28, 16)
        #    Pool      ->(?, 14, 14, 16)
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 16, kernel_size=1, stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=1 - keep_prob))

        # L4 FC 14x14x16 inputs -> 512 outputs
        self.fc1 = torch.nn.Linear(512, 512, bias=True)
        torch.nn.init.xavier_uniform(self.fc1.weight)
#         self.layer4 = torch.nn.Sequential(
#             self.fc1,
#             torch.nn.ReLU(),
#             torch.nn.Dropout(p=1 - keep_prob))
        # L5 Final FC 1024 inputs -> 512 outputs
        self.pool =  torch.nn.AdaptiveAvgPool2d((1,1))
        # Affine Layer
        self.Diag_Affine = Crude_Diag(512)
        # self.test_linear = torch.nn.Linear(512,512)
        self.fc2 = torch.nn.Linear(512, 4, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight) # initialize parameters
        # L6 Final FC 512 inputs -> 4 outputs
#         self.fc3 = torch.nn.Linear(512, 4, bias=True)
#         torch.nn.init.xavier_uniform_(self.fc3.weight) # initialize parameters

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x= x.reshape(x.shape[0],1,x.shape[1],x.shape[2])
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        # out = self.layer4(out)
#         print(out.size())
        # out = out.view(out.size(0), -1)
        # out = out.mean(axis = -1)
        out = torch.mean(out, axis=-1).reshape(-1,512)
#         print(out.size())# Flatten them for FC
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.Diag_Affine(out)
        # out = self.test_linear(out)
        out = self.fc2(out)
#         out = self.fc3(out)
        return out
    

In [39]:
    model = CNN()
    input = torch.randn(64,13,33)
    out = model(input)
    print(out)

tensor([[-0.0328, -0.2532, -0.0147, -0.5407],
        [ 0.4180, -0.3902, -0.8919, -0.1810],
        [-1.0130, -0.0062, -0.0935,  0.1081],
        [ 0.1796,  0.1419, -0.6890, -0.5396],
        [-1.0630,  0.1698, -0.4283, -0.7475],
        [-0.5486, -0.1372, -0.0305, -0.6501],
        [-0.0423, -0.3204, -0.3573, -0.7357],
        [ 0.2947, -0.1449, -0.2917, -0.4626]], grad_fn=<AddmmBackward>)


In [ ]:
model = CNN()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)     

print("network before turning off sparse layer", count_parameters(model))

for param in model.Diag_Affine.parameters():
    param.requires_grad = False     
print("network after turning off sparse layer", count_parameters(model))
model = CNN()
model.to('cuda')

In [ ]:
print(model)

In [ ]:
anger = 0
happiness = 0
sadness = 0
neutral = 0

for _, target in trainloader:
    labels = list(target.numpy())
    anger += len([x for x in labels if x == 0])
    happiness += len([x for x in labels if x == 1])
    sadness += len([x for x in labels if x == 2])
    neutral += len([x for x in labels if x == 3])
    
for _, target in valloader:
    labels = list(target.numpy())
    anger += len([x for x in labels if x == 0])
    happiness += len([x for x in labels if x == 1])
    sadness += len([x for x in labels if x == 2])
    neutral += len([x for x in labels if x == 3])

In [ ]:
label_to_class = {0: 'anger', 1: 'happiness', 2: 'sadness', 3: 'neutral'}
final_labels = ['anger', 'happiness', 'sadness', 'neutral']

sample_weights = [1/anger, 1/happiness, 1/sadness, 1/neutral]
print("anger {} happiness {} sadness {} neutral {}".format(anger, happiness, sadness, neutral))

class_weights = torch.FloatTensor(sample_weights).cuda()
print(class_weights)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# device ='cuda'
# model = model.to(device)

In [ ]:
# total_params = sum(p.numel() for p in model.parameters())
# print(f'{total_params:,} total parameters.')
# total_trainable_params = sum(
#     p.numel() for p in model.parameters() if p.requires_grad)
# print(f'{total_trainable_params:,} training parameters.')

In [ ]:
def plot_confusion_matrix(actual, predicted, labels):
    cm = confusion_matrix(actual, predicted, labels)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, annot_kws={"size": 10}, fmt='.0f'); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels);
    plt.show()

In [ ]:
# # instantiation
device='cuda'

# model = CNN().to(device)
# print("network after turning off sparse layer", count_parameters(net))


In [ ]:
def train(epoch, model):
    model.train()
    correct_train = 0
    train_loss = 0
    train_acc = 0
    
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        print(data.shape)
        
        # zero the gradient, forward, backward and running pytorch rhythm
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        # get the label of prediction
        pred = torch.max(output.data, 1)[1]
        correct_train += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\n'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    
    train_loss /= len(trainloader.dataset)
    train_acc = 100. * correct_train / len(trainloader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(train_loss))
    print('\nTrain Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct_train, len(trainloader.dataset), 100. * correct_train / len(trainloader.dataset)))
    
    return train_loss, int(train_acc.numpy())
            

In [ ]:
from sklearn.metrics import confusion_matrix
def test(model):
    model.eval()
    test_loss = 0
    test_acc = 0
    correct = 0
    history_test = []

    pred_model = []
    actual = []

    for data, target in valloader:
        data, target = data.to(device), target.to(device)

        # output from model
        output = model(data)

        # sum total loss
        test_loss += criterion(output, target).item()

        # get the label of prediction
        pred = torch.max(output.data, 1)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        pred_model.append(pred.cpu().numpy())
        actual.append(target.data.cpu().numpy())


    test_loss /= len(valloader.dataset)
    test_acc = 100. * correct / len(valloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valloader.dataset),
        100. * correct / len(valloader.dataset)))


    pred_with_label = [label_to_class[label] for label in list(np.concatenate(pred_model))]
    actual_with_label = [label_to_class[label] for label in list(np.concatenate(actual))]

    confusion_matrix(actual_with_label, pred_with_label, labels=final_labels)

    print('\n Classification Report \n {} \n'.format(classification_report(actual_with_label, pred_with_label)))

    return test_loss, int(test_acc.numpy())

In [ ]:
history, n_epoch = [], 101

for epoch in range(1, n_epoch):    
    # exp_lr_scheduler.step(epoch)
    # import pdb
    # pdb.set_trace()
    train_loss, train_acc = train(epoch, model)
    test_loss, test_acc = test(model)
    
    # plateau_scheduler.step(test_loss)
    history.append([train_loss, train_acc, test_loss, test_acc])

In [ ]:
history_df = pd.DataFrame(history, columns=["train_loss", "train_acc", "test_loss", "test_acc"])
history_df["epoch"] = [x for x in range(1, n_epoch)]
print(history_df)

In [ ]:
sns.set()
sns.lineplot(x='epoch', y='train_loss', data=history_df, color='b')
# plt.xticks(history_df.epoch)
sns.lineplot(x='epoch', y='test_loss', data=history_df, color='g')
plt.ylabel("loss")
plt.show()

In [ ]:
sns.set()
sns.lineplot(x='epoch', y='train_acc', data=history_df, color='b')
# plt.xticks(history_df.epoch)
sns.lineplot(x='epoch', y='test_acc', data=history_df, color='g')
plt.show()